<a href="https://colab.research.google.com/github/jmcaussade/Homologacion-Car-Makers/blob/main/Estandarizacion_marcas_modelos_Vin_dic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import os


In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import files

In [42]:
ruta_data_set = '/content/drive/MyDrive/Xinergy/Proyecto_homologacion_autos/database.xlsx'
df = pd.read_excel(ruta_data_set, sheet_name='Anexo 6')
df.head()
df

,Unnamed: 0,numcaso,tipo_vehiculo,fecha_servicio,plate,serie,model,Tipo de Auto,car_brand,car_model,car_type,Peso kg
0,NaN,355845,Auto,2023-09-01 00:01:13,SZD861A,3HGCM66563G001615,2003.0,estandar,Honda,ACCORD,Auto,NaN
1,NaN,355847,Camion,2023-09-01 00:06:09,TP9498G,3N6DD25T2EK005444,2014.0,estandar,NISSAN,ESTACAS,camion,NaN
2,NaN,355849,Auto,2023-09-01 00:08:38,NCZ1892,MEX5G2603JT120368,2018.0,estandar,Volkswagen,VENTO,Auto,NaN
3,NaN,355857,Auto,2023-09-01 00:24:34,FCJ597B,1C4BJWEG6FL629655,2015.0,estandar,JEEP,J WRANGLER,Auto,NaN
4,NaN,355864,Auto,2023-09-01 00:46:09,NJA2747,MA6CB6CD6KT029772,2019.0,estandar,CHEVROLET,BEAT,Auto,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
279424,NaN,745808,Equipo Ligero,2024-08-31 00:00:00,TDY040B,3N1CN7AD7KK399688,2019.0,NaN,NISSAN,VERSA,Equipo Ligero,NaN
279425,NaN,745810,Equipo Ligero,2024-08-31 00:00:00,JUB4510,LSJA24U99RN010717,2024.0,NaN,MG,HS,Equipo Ligero,NaN
279426,NaN,745811,Equipo Ligero,2024-08-31 00:00:00,RTR920A,3G1SF21X97S112886,2007.0,NaN,CHEVROLET,CHEVY,Equipo Ligero,NaN
279427,NaN,745814,Equipo Ligero,2024-08-31 00:00:00,PWEE63E,LJD5AA194R0124080,2024.0,NaN,KIA,SELTOS,Equipo Ligero,NaN


In [43]:
vin_dict = {
    # Acura
    "19U": "Acura", "2HH": "Acura", "5J6": "Acura",
    # Alfa Romeo
    "ZAR": "Alfa Romeo",
    # Altamirano
    "A11": "Altamirano",  # Assuming a custom manufacturer
    # Alubique
    "A12": "Alubique",  # Assuming a custom manufacturer
    # Astro
    "1A3": "Astro",  # Assuming a custom manufacturer
    # Audi
    "WA1": "Audi", "WAU": "Audi", "WVG": "Audi",
    # Austin
    "SAX": "Austin",
    # BAIC
    "LBG": "BAIC",
    # Bayage
    "1B7": "Bayage",  # Assuming a custom manufacturer
    # Beat
    "1BX": "Beat",  # Assuming a custom manufacturer
    # Benedeti
    "1B8": "Benedeti",  # Assuming a custom manufacturer
    # Bengala
    "1B9": "Bengala",  # Assuming a custom manufacturer
    # Bento
    "1BA": "Bento",  # Assuming a custom manufacturer
    # BMW
    "WBA": "BMW", "WBS": "BMW M", "WBX": "BMW",
    "5UX": "BMW", "4US": "BMW", "WBY": "BMW i",
    # Buick
    "1G4": "Buick", "1G5": "GMC", "1G6": "Cadillac", "1GB": "Buick",
    # BYD
    "LJ1": "BYD", "LJ2": "BYD", "LJ3": "BYD",
    # Cadillac
    "1G6": "Cadillac", "1G1": "Chevrolet",
    # Can-Am
    "2B3": "Can-Am",
    # Catrimoto
    "1C4": "Catrimoto",
    # Cavalier
    "1C6": "Cavalier",
    # Changan
    "LGW": "Changan", "LGS": "Changan",
    # Cupra
    "VSS": "Cupra",
    # Daewoo
    "KL1": "Daewoo", "KL2": "Daewoo", "KL3": "Daewoo", "KL4": "Chevrolet",
    # Dodge
    "1B3": "Dodge", "2B3": "Dodge", "3B3": "Dodge",
    # Ducati
    "ZDM": "Ducati",
    # Duke
    "1C5": "Duke",  # Assuming a custom manufacturer
    # Dynamo
    "1D5": "Dynamo",  # Assuming a custom manufacturer
    # Eco
    "2A4": "Eco",  # Assuming a custom manufacturer
    # Fiat
    "3C3": "Fiat", "ZFA": "Fiat", "ZFB": "Fiat", "ZFC": "Fiat",
    # Ford
    "1FA": "Ford", "1FB": "Ford", "1FC": "Ford", "1FD": "Ford",
    "1FM": "Ford", "1FT": "Ford", "1FU": "Ford", "1FV": "Ford",
    "2FA": "Ford", "2FB": "Ford", "2FC": "Ford", "2FD": "Ford",
    "2FM": "Ford", "2FT": "Ford", "2FU": "Ford", "2FV": "Ford",
    "3FA": "Ford", "3FB": "Ford", "3FC": "Ford", "3FD": "Ford",
    "3FM": "Ford", "3FT": "Ford", "3FU": "Ford", "3FV": "Ford",
    # Freightliner
    "1FV": "Freightliner", "4FV": "Freightliner",
    # General Motors
    "1G1": "Chevrolet", "1G2": "Pontiac", "1G3": "Oldsmobile", "1G4": "Buick",
    "1G5": "GMC", "1G6": "Cadillac", "2G1": "Chevrolet", "2G2": "Pontiac",
    # GMC
    "1G5": "GMC", "1G6": "Cadillac", "2G5": "GMC", "2G6": "Cadillac",
    # Great Wall
    "LWB": "Great Wall", "LWR": "Great Wall",
    # Harley Davidson
    "1HD": "Harley Davidson",
    # Haval
    "LHT": "Haval", "LHW": "Haval",
    # Honda
    "1HG": "Honda", "2HG": "Honda", "3HG": "Honda",
    "JHM": "Honda", "JHL": "Honda", "JHM": "Honda",
    "JHN": "Honda", "JHR": "Honda", "JHT": "Honda",
    "JHW": "Honda", "JHY": "Honda", "JHZ": "Honda",
    # Hummer
    "5GR": "Hummer",
    # Hyundai
    "5NP": "Hyundai", "5NM": "Hyundai", "5NMS": "Hyundai",
    "KM8": "Hyundai", "KMH": "Hyundai", "KMF": "Hyundai",
    "KMT": "Hyundai", "KMX": "Hyundai",
    # Infiniti
    "JNK": "Infiniti", "JN1": "Infiniti",
    # Isuzu
    "4S2": "Isuzu", "5S2": "Isuzu",
    # Italika
    "1I1": "Italika",
    # Jaguar
    "SAJ": "Jaguar",
    # Jeep
    "1J4": "Jeep", "1J8": "Jeep",
    # JMC
    "LFP": "JMC", "LGS": "JMC",
    # Kawasaki
    "JK1": "Kawasaki", "JK2": "Kawasaki",
    # Kenworth
    "1XK": "Kenworth",
    # Keeway
    "LFF": "Keeway",
    # Kia
    "KND": "Kia", "KNH": "Kia", "KNM": "Kia",
    "KNE": "Kia", "KNJ": "Kia", "KNP": "Kia",
    "KNR": "Kia", "KNS": "Kia", "KNT": "Kia",
    "KNU": "Kia", "KNV": "Kia", "KNX": "Kia",
    "KNA": "Kia",
    # KTM
    "VBK": "KTM",
    # Land Rover
    "SAL": "Land Rover", "SAD": "Land Rover",
    # Lexus
    "JTH": "Lexus", "JTJ": "Lexus",
    # Lincoln
    "5LM": "Lincoln",
    # Maserati
    "ZAM": "Maserati",
    # Mazda
    "JM1": "Mazda", "JM3": "Mazda", "4F2": "Mazda",
    "1YV": "Mazda", "3MZ": "Mazda", "JM7": "Mazda",
    "JM8": "Mazda", "JM9": "Mazda", "JMA": "Mazda",
    # Mercedes Benz
    "WDB": "Mercedes-Benz", "WDC": "Mercedes-Benz",
    "WDD": "Mercedes-Benz", "WDF": "Mercedes-Benz",
    "WDZ": "Mercedes-Benz", "WD3": "Mercedes-Benz",
    "WD4": "Mercedes-Benz", "WD5": "Mercedes-Benz",
    "WD6": "Mercedes-Benz", "WD7": "Mercedes-Benz",
    "WD8": "Mercedes-Benz", "WD9": "Mercedes-Benz",
    # Mercury
    "1M8": "Mercury",
    # MG
    "SA9": "MG",
    # Mini Cooper
    "WMW": "Mini Cooper",
    # Mitsubishi
    "JA4": "Mitsubishi", "JA7": "Mitsubishi", "1A4": "Mitsubishi",
    # Nissan
    "1N4": "Nissan", "1N6": "Nissan", "JN1": "Nissan",
    "JN3": "Nissan", "JN6": "Nissan", "JN8": "Nissan",
    "JNA": "Nissan", "JNC": "Nissan", "JNF": "Nissan",
    "JNK": "Nissan", "JNL": "Nissan", "JNM": "Nissan",
    "JNR": "Nissan", "JNT": "Nissan", "JNV": "Nissan",
    # Peugeot
    "VF3": "Peugeot", "VFW": "Peugeot",
    # Piaggio
    "ZAP": "Piaggio",
    # Plymouth
    "1P3": "Plymouth",
    # Pontiac
    "1G2": "Pontiac", "1G3": "Pontiac",
    # Porsche
    "WP0": "Porsche",
    # Renault
    "VF1": "Renault", "VFS": "Renault",
    # Saab
    "YS3": "Saab",
    # Scania
    "YS3": "Scania",
    # Seat
    "VSS": "Seat",
    # Subaru
    "JF1": "Subaru", "JF2": "Subaru", "JF3": "Subaru",
    "JF4": "Subaru", "JF5": "Subaru", "JF6": "Subaru",
    "JF7": "Subaru", "JF8": "Subaru", "JF9": "Subaru",
    # Suzuki
    "2S3": "Suzuki",
    # Tesla
    "5YJ": "Tesla", "7SA": "Tesla", "7SB": "Tesla",
    "7SC": "Tesla", "7SD": "Tesla", "7SE": "Tesla",
    "7SF": "Tesla", "7SG": "Tesla", "7SH": "Tesla",
    # Tiggo
    "LBG": "Tiggo",
    # Toyota
    "1NX": "Toyota", "2T1": "Toyota", "4T1": "Toyota",
    "5TD": "Toyota", "JTD": "Toyota", "JT2": "Toyota",
    "JT3": "Toyota", "JT4": "Toyota", "JT5": "Toyota",
    "JT6": "Toyota", "JT7": "Toyota", "JT8": "Toyota",
    "JTE": "Toyota", "JTF": "Toyota", "JTH": "Toyota",
    "JTL": "Toyota", "JTM": "Toyota", "JTN": "Toyota",
    "JTP": "Toyota", "JTU": "Toyota", "JTV": "Toyota",
    # Vespa
    "ZAP": "Vespa",
    # Volkswagen
    "3VW": "Volkswagen", "9BW": "Volkswagen",
    "WVW": "Volkswagen", "WV1": "Volkswagen Commercial",
    "WV2": "Volkswagen Commercial", "WV3": "Volkswagen Commercial",
    "WV4": "Volkswagen Commercial", "WV5": "Volkswagen Commercial",
    "WV6": "Volkswagen Commercial", "WV7": "Volkswagen Commercial",
    "WV8": "Volkswagen Commercial", "WV9": "Volkswagen Commercial",
    # Volvo
    "YV1": "Volvo", "YV2": "Volvo",
    # Yamaha
    "JYA": "Yamaha",
}




In [21]:
# vin_dict = {
#     # Ford
#     "1FA": "Ford", "1FB": "Ford", "1FC": "Ford", "1FD": "Ford",
#     "1FM": "Ford", "1FT": "Ford", "1FU": "Ford", "1FV": "Ford",
#     "2FA": "Ford", "2FB": "Ford", "2FC": "Ford", "2FD": "Ford",
#     "2FM": "Ford", "2FT": "Ford", "2FU": "Ford", "2FV": "Ford",
#     "3FA": "Ford", "3FB": "Ford", "3FC": "Ford", "3FD": "Ford",
#     "3FM": "Ford", "3FT": "Ford", "3FU": "Ford", "3FV": "Ford",
#     # General Motors (Chevrolet, GMC, Cadillac, Buick)
#     "1G1": "Chevrolet", "1G2": "Pontiac", "1G3": "Oldsmobile",
#     "1G4": "Buick", "1G5": "GMC", "1G6": "Cadillac",
#     "2G1": "Chevrolet", "2G2": "Pontiac", "2G3": "Oldsmobile",
#     "2G4": "Buick", "2G5": "GMC", "2G6": "Cadillac",
#     "3G1": "Chevrolet", "3G2": "Pontiac", "3G3": "Oldsmobile",
#     "3G4": "Buick", "3G5": "GMC", "3G6": "Cadillac",
#     # Toyota
#     "1NX": "Toyota", "2T1": "Toyota", "4T1": "Toyota",
#     "5TD": "Toyota", "JTD": "Toyota", "JT2": "Toyota",
#     "JT3": "Toyota", "JT4": "Toyota", "JT5": "Toyota",
#     "JT6": "Toyota", "JT7": "Toyota", "JT8": "Toyota",
#     "JTE": "Toyota", "JTF": "Toyota", "JTH": "Toyota",
#     "JTL": "Toyota", "JTM": "Toyota", "JTN": "Toyota",
#     "JTP": "Toyota", "JTU": "Toyota", "JTV": "Toyota",
#     # Honda
#     "1HG": "Honda", "2HG": "Honda", "3HG": "Honda",
#     "JHM": "Honda", "JHL": "Honda", "JHM": "Honda",
#     "JHN": "Honda", "JHR": "Honda", "JHT": "Honda",
#     "JHW": "Honda", "JHY": "Honda", "JHZ": "Honda",
#     # Nissan
#     "1N4": "Nissan", "1N6": "Nissan", "JN1": "Nissan",
#     "JN3": "Nissan", "JN6": "Nissan", "JN8": "Nissan",
#     "JNA": "Nissan", "JNC": "Nissan", "JNF": "Nissan",
#     "JNK": "Nissan", "JNL": "Nissan", "JNM": "Nissan",
#     "JNR": "Nissan", "JNT": "Nissan", "JNV": "Nissan",
#     # BMW
#     "WBA": "BMW", "WBS": "BMW M", "WBX": "BMW",
#     "5UX": "BMW", "4US": "BMW", "WBY": "BMW i",
#     # Mercedes-Benz
#     "WDB": "Mercedes-Benz", "WDC": "Mercedes-Benz",
#     "WDD": "Mercedes-Benz", "WDF": "Mercedes-Benz",
#     "WDZ": "Mercedes-Benz", "WD3": "Mercedes-Benz",
#     "WD4": "Mercedes-Benz", "WD5": "Mercedes-Benz",
#     "WD6": "Mercedes-Benz", "WD7": "Mercedes-Benz",
#     "WD8": "Mercedes-Benz", "WD9": "Mercedes-Benz",
#     # Volkswagen
#     "3VW": "Volkswagen", "9BW": "Volkswagen",
#     "WVW": "Volkswagen", "WV1": "Volkswagen Commercial",
#     "WV2": "Volkswagen Commercial", "WV3": "Volkswagen Commercial",
#     "WV4": "Volkswagen Commercial", "WV5": "Volkswagen Commercial",
#     "WV6": "Volkswagen Commercial", "WV7": "Volkswagen Commercial",
#     "WV8": "Volkswagen Commercial", "WV9": "Volkswagen Commercial",
#     # Hyundai
#     "5NP": "Hyundai", "5NM": "Hyundai", "5NMS": "Hyundai",
#     "KM8": "Hyundai", "KMH": "Hyundai", "KMF": "Hyundai",
#     "KMT": "Hyundai", "KMX": "Hyundai", "KNA": "Kia",
#     # Kia
#     "KND": "Kia", "KNH": "Kia", "KNM": "Kia",
#     "KNE": "Kia", "KNJ": "Kia", "KNP": "Kia",
#     "KNR": "Kia", "KNS": "Kia", "KNT": "Kia",
#     "KNU": "Kia", "KNV": "Kia", "KNX": "Kia",
#     # Subaru
#     "JF1": "Subaru", "JF2": "Subaru", "JF3": "Subaru",
#     "JF4": "Subaru", "JF5": "Subaru", "JF6": "Subaru",
#     "JF7": "Subaru", "JF8": "Subaru", "JF9": "Subaru",
#     # Mazda
#     "JM1": "Mazda", "JM3": "Mazda", "4F2": "Mazda",
#     "1YV": "Mazda", "3MZ": "Mazda", "JM7": "Mazda",
#     "JM8": "Mazda", "JM9": "Mazda", "JMA": "Mazda",
#     # Tesla
#     "5YJ": "Tesla", "7SA": "Tesla", "7SB": "Tesla",
#     "7SC": "Tesla", "7SD": "Tesla", "7SE": "Tesla",
#     "7SF": "Tesla", "7SG": "Tesla", "7SH": "Tesla",
#     # Fiat
#     "3C3": "Fiat", "ZFA": "Fiat", "ZFB": "Fiat"
# }

In [44]:
df["clean_serie"] = df["serie"].fillna("").astype(str).str.strip()


In [45]:
def categorize_by_vin(vin, vin_dict):
    vin_prefix = vin[:3]  # Extract the first three characters
    return vin_dict.get(vin_prefix, "Unknown")  # Get the maker or default to "Unknown"

In [46]:
# Create a new column for the categorized maker
df["maker_vin"] = df["clean_serie"].apply(lambda x: categorize_by_vin(x, vin_dict))

In [47]:
df

,Unnamed: 0,numcaso,tipo_vehiculo,fecha_servicio,plate,serie,model,Tipo de Auto,car_brand,car_model,car_type,Peso kg,clean_serie,maker_vin
0,NaN,355845,Auto,2023-09-01 00:01:13,SZD861A,3HGCM66563G001615,2003.0,estandar,Honda,ACCORD,Auto,NaN,3HGCM66563G001615,Honda
1,NaN,355847,Camion,2023-09-01 00:06:09,TP9498G,3N6DD25T2EK005444,2014.0,estandar,NISSAN,ESTACAS,camion,NaN,3N6DD25T2EK005444,Unknown
2,NaN,355849,Auto,2023-09-01 00:08:38,NCZ1892,MEX5G2603JT120368,2018.0,estandar,Volkswagen,VENTO,Auto,NaN,MEX5G2603JT120368,Unknown
3,NaN,355857,Auto,2023-09-01 00:24:34,FCJ597B,1C4BJWEG6FL629655,2015.0,estandar,JEEP,J WRANGLER,Auto,NaN,1C4BJWEG6FL629655,Catrimoto
4,NaN,355864,Auto,2023-09-01 00:46:09,NJA2747,MA6CB6CD6KT029772,2019.0,estandar,CHEVROLET,BEAT,Auto,NaN,MA6CB6CD6KT029772,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279424,NaN,745808,Equipo Ligero,2024-08-31 00:00:00,TDY040B,3N1CN7AD7KK399688,2019.0,NaN,NISSAN,VERSA,Equipo Ligero,NaN,3N1CN7AD7KK399688,Unknown
279425,NaN,745810,Equipo Ligero,2024-08-31 00:00:00,JUB4510,LSJA24U99RN010717,2024.0,NaN,MG,HS,Equipo Ligero,NaN,LSJA24U99RN010717,Unknown
279426,NaN,745811,Equipo Ligero,2024-08-31 00:00:00,RTR920A,3G1SF21X97S112886,2007.0,NaN,CHEVROLET,CHEVY,Equipo Ligero,NaN,3G1SF21X97S112886,Unknown
279427,NaN,745814,Equipo Ligero,2024-08-31 00:00:00,PWEE63E,LJD5AA194R0124080,2024.0,NaN,KIA,SELTOS,Equipo Ligero,NaN,LJD5AA194R0124080,Unknown


In [48]:
# Filter rows where the "maker" column is "Unknown"
unknown_maker_rows = df[df["maker_vin"] != "Unknown"]

# Print the filtered rows
print(unknown_maker_rows)


        Unnamed: 0  numcaso  tipo_vehiculo      fecha_servicio     plate  \
0              NaN   355845           Auto 2023-09-01 00:01:13   SZD861A   
3              NaN   355857           Auto 2023-09-01 00:24:34   FCJ597B   
5              NaN   355870           Auto 2023-09-01 00:57:06    FF935B   
11             NaN   355887           Auto 2023-09-01 03:07:14   RXP245B   
12             NaN   355889         Camion 2023-09-01 04:47:29  1PH3414B   
...            ...      ...            ...                 ...       ...   
279413         NaN   745758  Equipo Ligero 2024-08-31 00:00:00   JHM3911   
279414         NaN   745766  Equipo Ligero 2024-08-31 00:00:00   JRC6113   
279416         NaN   745785  Equipo Ligero 2024-08-31 00:00:00   B53SKN5   
279417         NaN   745788  Equipo Ligero 2024-08-31 00:00:00   B53SKN5   
279423         NaN   745807  Equipo Ligero 2024-08-31 00:00:00   TDJ607B   

                    serie   model Tipo de Auto   car_brand    car_model  \
0       3HGC

#Descarga archivo


In [49]:
file_name = 'dataframe_makers_dict.xlsx'

In [ ]:
#Descarga en Colab session

df.to_excel(file_name, index=False)

# Confirm the export (optional)
print(f"DataFrame exported successfully to {file_name}")

In [50]:
#Para descargar localmente
df.to_excel(file_name, index=False)
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>